# Using Embeddings for unsupervised clustering with named clusters (and other fun things)

In this notebook, we use a subset of the [Top 10000 Popular Movies Dataset](https://www.kaggle.com/datasets/db55ac3dfd0098a0cf96dd542807f9253a16587ff233e06baef372bccfd09942) to calculate embeddings on movie descriptions and then apply kmeans to find similar clusters. Once we have these clusters, we'll use a prompt to extract the topics from each cluster. 

In [ ]:
import os
import tiktoken
import openai
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from openai.embeddings_utils import cosine_similarity
from tenacity import retry, wait_random_exponential, stop_after_attempt


# Load environment variables
load_dotenv('../.env')

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = os.getenv('OPENAI_API_VERSION', "2022-12-01")
OPENAI_API_BASE = openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")

# Define embedding model and encoding
EMBEDDING_MODEL = os.getenv('OPENAI_EMBEDDING_MODEL', 'text-embedding-ada-002')
EMBEDDING_ENCODING = os.getenv('OPENAI_EMBEDDING_ENCODING', 'cl100k_base')
EMBEDDING_CHUNK_SIZE = os.getenv('OPENAI_EMBEDDING_CHUNK_SIZE', 8000)
COMPLETION_MODEL = os.getenv('OPENAI_COMPLETION_MODEL', 'gpt-35-turbo')

# initialize tiktoken for encoding text
encoding = tiktoken.get_encoding(EMBEDDING_ENCODING)

params_gathered = dict(
    EMBEDDING_MODEL=EMBEDDING_MODEL,
    EMBEDDING_ENCODING=EMBEDDING_ENCODING,
    EMBEDDING_CHUNK_SIZE=EMBEDDING_CHUNK_SIZE,
    COMPLETION_MODEL=COMPLETION_MODEL,
    OPENAI_API_VERSION=openai.api_version,
    OPENAI_API_BASE=OPENAI_API_BASE
)
for key, val in params_gathered.items():
    print(key, val)

Load `movies.csv`:

In [ ]:
df = pd.read_csv('../../data/movies/movies.csv')
print(df.shape)
df.head()

Next, let's create a new column and calculate how many tokens each embedding would cost. This allows us to get an estimate how much we'd pay to create embeddings on the whole dataset.

In [ ]:
# add a new column to the dataframe where you put the token count of the review
df = df.assign(token_count=df['overview'].apply(lambda x: len(encoding.encode(x))))

# print the first 5 rows of the dataframe, then also the total number of tokens
total_tokens = df['token_count'].sum()

cost_for_embeddings = total_tokens / 1000 * 0.0004
print(f"Test would cost ${cost_for_embeddings} for embeddings")

Let's define our embedding method. Please note the use of tenacity for having an automated retry mechanism, in case we hit the TPS limits of Azure OpenAI Service.

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(1))
def get_embedding(text) -> list[float]:
    text = text.replace("\n", " ")
    return openai.Embedding.create(input=text, engine=EMBEDDING_MODEL)["data"][0]["embedding"]

Let's creating the embeddings:

In [ ]:
#Create a small test call to verify connection:
text = "Hello World!"
e = openai.Embedding.create(input=text, engine=EMBEDDING_MODEL)["data"][0]["embedding"]
print(f'Testing connection returned embedding with length {len(e)}')
#Now let's embed our data. Notice that this needs internet connection and consumes tokens/money.
#If this fails, try using a local embedder function.
df = df.assign(embedding=df['overview'].apply(lambda x: get_embedding(x)))
df.head()

Next, let's create clusters on the embeddings using KMeans. In this case, we'll go for 5 clusters, knowing that this might be wrong.

In [ ]:
# train k-means on df embeddings
from sklearn.cluster import KMeans

n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, init="k-means++", random_state=42)
kmeans.fit(df['embedding'].to_list())
df = df.assign(cluster=kmeans.labels_)
df.head()

Now that we have a cluster per row, let's use t-SNE to project our embeddings into 2d space and visualize the clusters.

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(
    n_components=2, perplexity=50, random_state=42, init="random", learning_rate=200
)

matrix = np.vstack(df.embedding.values)
print(matrix.shape)
vis_dims2 = tsne.fit_transform(matrix)

x = [x for x, y in vis_dims2]
y = [y for x, y in vis_dims2]

for category, color in enumerate(["purple", "green", "red", "blue","yellow", 'black', 'orange', 'brown', 'pink', 'grey']):
    xs = np.array(x)[df.cluster == category]
    ys = np.array(y)[df.cluster == category]
    plt.scatter(xs, ys, color=color, alpha=0.3)

    avg_x = xs.mean()
    avg_y = ys.mean()

    plt.scatter(avg_x, avg_y, marker="x", color=color, s=100)

Ugh ok, that does not look great, but was somehow expected. We have all kinds of movies and with only 5 clusters, this might not be ideal. However, if you look closely, you can make up some rough shape that resembles a cluster. Also, movies might fall into two or more categories, so it kind of makes sense.

Lastly, let's take a few examples from each cluster, send them to OpenAI and get the common theses extracted:

In [ ]:
# take 10 movies from each cluster and write a prompt that asks what these have in common
# ideally you would use more movies than 10, but this is just a demo
for i in range(n_clusters):
    reviews = df[df['cluster'] == i]['overview'].sample(10)
    reviews = "\n".join(reviews.values.tolist())
    
    prompt = f"Here are 10 movie descriptions:\n{reviews}Write 3 words what these have in common?"
    #print(prompt)
    response = openai.Completion.create(engine=COMPLETION_MODEL, prompt=prompt, temperature=0.7, max_tokens=100, top_p=1, frequency_penalty=0, presence_penalty=0, stop=None)['choices'][0]['text'].strip()
    print(f"Cluster {i} topics: {response}")
    movies = df[df['cluster'] == i]['original_title'].sample(25)
    print(f"Movies from cluster {i}: {', '.join(movies.values.tolist())}")
    print("================")    

Doesn't look too bad. But again, 10 samples for each class might not be enough, given the low number of clusters. Anyway, looking at the movie titles, some of the topics actually make fairly ok sense.

# Bonus: Using the embeddings to build a simple recommendation system

Another thing we can do is use the embeddings for building a very simple recommendation system. So let's try it:

In [ ]:
# Let's pick a movie that exists in df, keeping in mind we only have 500 movies in it!
movie = "Frozen"

# get embedding for movie
e = df[df['original_title'] == movie]['embedding'].values[0]

# get cosine similarity between movie and all other movies and sort ascending
similarities = df['embedding'].apply(lambda x: cosine_similarity(x, e))

# get most similar movies
movies = df.assign(similarity=similarities).sort_values(by='similarity', ascending=False)[['original_title', 'similarity', 'overview']]
movies[0:6]

Besides the last one, this actually does not look too bad...probably would have been useful if we added movie categories and age ratings to our recommendations... :)